<a href="https://colab.research.google.com/github/adidam/rag-impl/blob/main/Real_World_RAG_Impl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install datasets
!pip install sentence-transformers
!pip install rank-bm25
!pip install torch transformers
!pip install huggingface_hub
!pip install langchain-community
!pip install accelerate
!pip install rank-bm25 nltk
!pip install scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 wh

## **Adding the Imports**

In [4]:
import numpy as np
import os
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import random_split
import torch.optim as optim
import matplotlib.pyplot as plt
import nltk

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

## **Loading the RAGBench Dataset**

 ## **Chunking the Dataset**

In [5]:
# embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = "BAAI/LLM-Embedder"
# embedding_model = "BAAI/bge-large-en"

# **Small to Big Chunking**

In [6]:
# New code - 12/4 10 pm

from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(embedding_model)
#tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

# Sliding window configuration
TOKEN_LIMIT = 512
SLIDING_WINDOW_OVERLAP = 100  # Overlap between consecutive chunks (in tokens)

# Function for chunking with token limit and sliding window
def chunk_with_token_limit(text, token_limit=512, overlap=100):
    sentences = sent_tokenize(text)  # Split text into sentences
    chunks = []  # Store resulting chunks
    current_chunk = []  # Temporarily hold sentences for the current chunk
    current_chunk_tokens = 0  # Token count for the current chunk

    for sentence in sentences:
        # Tokenize the sentence and calculate its token count
        sentence_tokens = tokenizer.tokenize(sentence)
        num_tokens = len(sentence_tokens)

        # print(f"Tokens: {sentence_tokens[0]}")

        # If adding this sentence exceeds the token limit
        if current_chunk_tokens + num_tokens > token_limit:
            # Save the current chunk
            chunk_text = " ".join(current_chunk)
            chunks.append(chunk_text)

            # Prepare the next chunk with overlap
            overlap_tokens = tokenizer.tokenize(" ".join(current_chunk[-1:]))
            current_chunk = [sentence for sentence in current_chunk[-(overlap // len(overlap_tokens)) :]] if current_chunk else []
            current_chunk_tokens = sum(len(tokenizer.tokenize(sent)) for sent in current_chunk)

        # Add the sentence to the current chunk
        current_chunk.append(sentence)
        current_chunk_tokens += num_tokens

    # Add the last chunk if it exists
    if current_chunk:
        chunk_text = " ".join(current_chunk)
        chunks.append(chunk_text)

    return chunks

def process_document_with_identifiers(document):
    processed_data = []
    title_count = -1  # to start from 0
    # print("document>>>>>>>",document)
    for section in document:
        section_chunks = []
        passage_count = [ord('a')]  # Passage identifier as a list to handle nested increments
        title_count += 1  # Increment title count

        # Tokenize the section into sentences
        sentences = sent_tokenize(section)
        for sentence in sentences:
            if sentence.startswith("Title:"):
                # New document detected
                identifier = f"{title_count}{''.join(chr(c) for c in passage_count)}"  # Identifier for the title

                # Commented this line to integrate and test with small to big. To be uncommented after testing
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)

                #Added this line to replace sliding window tokenization with hybrid tokenization (sliding window + small-to-big)
                #chunked_texts = hybrid_chunking(sentence,tokenizer, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)

                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])
                passage_count = [ord('a')]  # Reset passage count for the new document
            else:
                # Sentence under the current document
                identifier = f"{title_count}{''.join(chr(c) for c in passage_count)}"

                # Commented this line to integrate and test with small to big. To be uncommented after testing
                chunked_texts = chunk_with_token_limit(sentence, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)

                #Added this line to replace sliding window tokenization with hybrid tokenization (sliding window + small-to-big)
                #chunked_texts = hybrid_chunking(sentence,tokenizer, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)
                print("chunked_texts>>>>process_document_with_identifiers>>>>> "+ "".join(chunked_texts))

                for chunk in chunked_texts:
                    section_chunks.append([identifier, chunk])

                # Increment passage_count intelligently
                i = len(passage_count) - 1
                while i >= 0:
                    passage_count[i] += 1
                    if passage_count[i] > ord('z'):
                        passage_count[i] = ord('a')
                        if i == 0:
                            passage_count.insert(0, ord('a'))  # Add a new character to the identifier
                        i -= 1
                    else:
                        break


        print("section_chunks>>>>>>>",section_chunks)
        processed_data.append(section_chunks)

    return processed_data

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/396 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [7]:
# Added by Venkat on 1/29
def small_to_big_chunking(sliding_chunks):
    """
    Implements small-to-big chunking on top of sliding window chunks.

    Args:
        sliding_chunks (list): List of small overlapping chunks from sliding window.
        tokenizer: Tokenizer for counting tokens.
        token_limit (int): Max token length for each chunk.

    Returns:
        List of progressively larger chunks.
    """
    # tokenizer obtained in earlier cells for  "BAAI/LLM-Embedder" embedder
    token_limit=512
    big_chunks = []

    for i in range(len(sliding_chunks)):
        merged_text = ""
        for j in range(i + 1):  # Combine previous chunks progressively
            merged_text += " " + sliding_chunks[j]
            token_count = len(tokenizer.encode(merged_text, add_special_tokens=False))
            if token_count > token_limit:  # Ensure it doesn't exceed token limit
                break
        big_chunks.append(merged_text.strip())

    return big_chunks


def hybrid_chunking(text, tokenizer, token_limit=512, overlap=100):
    """
    Combines sliding window chunking and small-to-big chunking.

    Args:
        text (str): Input text.
        tokenizer: Tokenizer for counting tokens.
        token_limit (int): Max token length for each chunk.
        overlap (int): Overlapping tokens between chunks.

    Returns:
        List of both sliding window and small-to-big chunks.
    """
    # Step 1: Get small sliding window chunks
    sliding_chunks = chunk_with_token_limit(text)

    print("sliding_chunks>>>> ",sliding_chunks)

    # Step 2: Generate small-to-big chunks
    big_chunks = small_to_big_chunking(sliding_chunks)

    # Combine both
    print("sliding_chunks + big_chunks >>>> "+sliding_chunks + big_chunks)
    return sliding_chunks + big_chunks


# From Shiv (not yet used currently)
def small_to_big_chunking(text):
    sentences = sent_tokenize(text)  # Start small: split into sentences
    chunks = []
    current_chunk = []
    current_tokens = 0
    token_limit = 512
    for sentence in sentences:
        sentence_tokens = tokenizer.tokenize(sentence)
        num_tokens = len(sentence_tokens)

        # If adding this sentence exceeds the token limit, finalize the current chunk
        if current_tokens + num_tokens > token_limit:
            chunks.append(" ".join(current_chunk))
            current_chunk = []  # Start a new chunk
            current_tokens = 0

        # Add sentence to the current chunk
        current_chunk.append(sentence)
        current_tokens += num_tokens

    # Add the last chunk
    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks


# **Load and view first 5 rows of the Dataset**

In [8]:
# Print the top 5 rows of the dataset, for debugging purpose, we only generate embeddings for these 5 rows
from datasets import load_dataset
datasets = ['techqa']
data = load_dataset("rungalileo/ragbench", datasets[0], split="train")
top_5_rows = data.select(range(5))

for i, row in enumerate(top_5_rows):
    print(f"Row {i + 1}:")
    for field, value in row.items():
        print(f"  {field}: {value}")
    print()

README.md:   0%|          | 0.00/24.7k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/22.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/5.40M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/5.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1192 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/304 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/314 [00:00<?, ? examples/s]

Row 1:
  id: techqa_TRAIN_Q337
  question: Why does the other instance of my multi-instance qmgr seem to hang after a failover? Queue manager will not start after failover.
  documents: ['HL083112 mqlpgrlg ZX000001 ExecCtrlrMain lpiRC_LOG_NOT_AVAILABLE mscs TECHNOTE (TROUBLESHOOTING)\n\nPROBLEM(ABSTRACT)\n You attempt to failover from the primary to secondary node under MSCS. Your WebSphere MQ queue manager fails to come up on the secondary node, and errors are generated. \n\nSYMPTOM\nThe sequence seen in the FDC files show:\n\n\nProbe Id :- HL083112 \nComponent :- mqlpgrlg \nProcess Name :- D:\\Programs\\MQSeries\\bin\\amqzxma0.exe \nMajor Errorcode :- hrcE_MQLO_UNEXPECTED_OS_ERROR \nMQM Function Stack\nkpiStartup\napiStartup\nalmPerformReDoPass\nhlgScanLogBegin\nmqlpgrlg\nxcsFFST\n\n\nProbe Id :- ZX000001\nComponent :- ExecCtrlrMain \nProcess Name :- D:\\Programs\\MQSeries\\bin\\amqzxma0.exe \nMajor Errorcode :- xecF_E_UNEXPECTED_RC \nMinor Errorcode :- lpiRC_LOG_NOT_AVAILABLE \nProb

## **Generate Embeddings**

In [9]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

#datasets = ['pubmedqa','tatqa', 'techqa'] # d4

datasets = ['techqa']

# Initialize storage for documents, IDs, and metadata
all_documents = []
all_ids = []
all_metadatas = []

# Process each dataset
doc_idx = 0  # Global document index for unique IDs
for dataset in datasets:
    data = load_dataset("rungalileo/ragbench", dataset, split="train")
    #only select first 5 records for debugging duplicate records. **PLEASE REMOVE THIS AFTER DEBUGGING**
    data = data.select(range(5))
    for idx, row in tqdm(enumerate(data), desc=f"Processing {dataset}"):
        # Extract document text
        doc_text = row.get('documents', '')

        # Skip if no documents found
        if not doc_text:
            continue

        # Process the document
        processed_output = process_document_with_identifiers(doc_text)
        added_item_idxs = set()

        # Populate the lists
        for section_idx, section in enumerate(processed_output):
            for item_idx, (prefix, content) in enumerate(section):
                # Skip if this item_idx has already been processed
                if item_idx in added_item_idxs:
                    continue

                # Add the item_idx to the set to track it
                added_item_idxs.add(item_idx)

                # Add the document
                document = f"[{prefix}] {content}"
                all_documents.append(document)

                # Construct a globally unique ID
                doc_id = f"{dataset}_{doc_idx}_{section_idx}_{item_idx}"
                all_ids.append(doc_id)

                # Construct metadata
                metadata = {
                    "dataset": dataset,
                    "global_index": doc_idx,
                    "section_index": section_idx,
                    "item_index": item_idx,
                    "prefix": prefix,
                    "type": "Title" if prefix.endswith("a") else "Passage",
                }
                all_metadatas.append(metadata)

        doc_idx += 1  # Increment global document index

# Step 4: Generate Embeddings
#embedder = SentenceTransformer("all-MiniLM-L6-v2")  # Pretrained sentence transformer
embedder = SentenceTransformer(embedding_model)  # Pretrained sentence transformer
batch_size = 2500  # Adjust based on available memory

# Generate embeddings in batches
all_embeddings = []
for i in tqdm(range(0, len(all_documents), batch_size), desc="Generating embeddings"):
    batch_docs = all_documents[i:i + batch_size]
    batch_embeddings = embedder.encode(batch_docs, show_progress_bar=True)
    all_embeddings.extend(batch_embeddings)

Processing techqa: 1it [00:00,  7.03it/s]

chunked_texts>>>>process_document_with_identifiers>>>>> HL083112 mqlpgrlg ZX000001 ExecCtrlrMain lpiRC_LOG_NOT_AVAILABLE mscs TECHNOTE (TROUBLESHOOTING)

PROBLEM(ABSTRACT)
 You attempt to failover from the primary to secondary node under MSCS.
chunked_texts>>>>process_document_with_identifiers>>>>> Your WebSphere MQ queue manager fails to come up on the secondary node, and errors are generated.
chunked_texts>>>>process_document_with_identifiers>>>>> SYMPTOM
The sequence seen in the FDC files show:


Probe Id :- HL083112 
Component :- mqlpgrlg 
Process Name :- D:\Programs\MQSeries\bin\amqzxma0.exe 
Major Errorcode :- hrcE_MQLO_UNEXPECTED_OS_ERROR 
MQM Function Stack
kpiStartup
apiStartup
almPerformReDoPass
hlgScanLogBegin
mqlpgrlg
xcsFFST


Probe Id :- ZX000001
Component :- ExecCtrlrMain 
Process Name :- D:\Programs\MQSeries\bin\amqzxma0.exe 
Major Errorcode :- xecF_E_UNEXPECTED_RC 
Minor Errorcode :- lpiRC_LOG_NOT_AVAILABLE 
Probe Description :- AMQ6118: An internal WebSphere MQ error 

Token indices sequence length is longer than the specified maximum sequence length for this model (839 > 512). Running this sequence through the model will result in indexing errors
Processing techqa: 5it [00:00, 13.56it/s]

chunked_texts>>>>process_document_with_identifiers>>>>> RESOLVING THE PROBLEM
C&DS development has confirmed these messages are noise and can be ignored.
chunked_texts>>>>process_document_with_identifiers>>>>> RELATED INFORMATION
 Need more help?
chunked_texts>>>>process_document_with_identifiers>>>>> Our C&DS forum is Live!
chunked_texts>>>>process_document_with_identifiers>>>>> [https://developer.ibm.com/answers/topics/collaboration.html?smartspace=predictive-analytics]
section_chunks>>>>>>> [['4a', ' TECHNOTE (TROUBLESHOOTING)\n\nPROBLEM(ABSTRACT)\n The Collaboration and Deployment Services (C&DS) setup.log is filled with errors when attempting to deploy C&DS 4.2 into WebLogic 11G application server.'], ['4b', 'SYMPTOM\nThere are many errors that are seen within our setup.log after attempting to deploy C&DS 4.2 into a WebLogic application server.'], ['4c', 'The setup reports that it completes.'], ['4d', 'However, the errors that are being seen would indicate the opposite.'], ['4e', 

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Generating embeddings:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 1/1 [00:51<00:00, 51.36s/it]


## **Store Embeddings into Milvus**

In [10]:
!pip install pymilvus pymilvus[model]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.4/222.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.70.0
    Uninstalling grpcio-1.70.0:
      Successfully uninstalled grpcio-1.70.0


# **Functions to check uniqueness of data being inserted to db**

In [11]:
import hashlib

# Function to generate a hash based on content and key metadata
def generate_hash(content, metadata):
    """Generate a unique hash for the document content and key metadata."""
    key_fields = f"{content}|{metadata.get('item_index')}|{metadata.get('prefix')}"
    return hashlib.md5(key_fields.encode('utf-8')).hexdigest()

# Function to retrieve existing hashes from the database
def get_existing_hashes(collection):
    """Retrieve all existing hashes (IDs) currently in the database."""
    all_records = collection.get(include=["documents", "metadatas"])  # Fetch documents and metadata
    existing_hashes = set()
    for doc, metadata in zip(all_records["documents"], all_records["metadatas"]):
        doc_hash = generate_hash(doc, metadata)
        existing_hashes.add(doc_hash)
    return existing_hashes

# Function to retrieve existing hashes from the database
def get_existing_hashes_milvus(all_records):
    """Retrieve all existing hashes (IDs) currently in the database."""
    existing_hashes = set()
    print(f"all records >>> {len(all_records)}")
    if all_records == None or len(all_records) == 0:
        return existing_hashes

    for record in all_records:
        doc = record.get("documents")
        metadata = record.get("metadata")
        doc_hash = generate_hash(doc, metadata)
        existing_hashes.add(doc_hash)
    return existing_hashes

In [12]:
import numpy as np
from pymilvus import connections
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection
from pymilvus import MilvusClient
from pymilvus import utility

class VectorDataStore:
    db_url = "http://localhost:19530"

    #description = f"collection created for {self.name}"

    def __init__(self, path="/content/ragbench.db"):
        self.client = MilvusClient(path)


    def get_or_create_collection(self, name, vec_dim=128):
        try:
            self.get_collection(name)
        except:
            print(f"Collection {name} doesn't exist. Creating...")
            self.create_collection(name, vec_dim)


    def create_collection(self, name, vec_dim=128):
        if self.client.has_collection(name):
            self.default_collection_name = name

        self.description = f"collection to store {name}"

        index_params = self.client.prepare_index_params()
        index_params.add_index(
            field_name="embedding",
            index_type="AUTOINDEX",
            metric_type="COSINE"
        )
        index_params.add_index(
            field_name="sparse",
            index_type="SPARSE_INVERTED_INDEX",
            metric_type="IP"
        )
        schema = self.client.create_schema(
            auto_id=False,
            enable_dynamic_fields=True,
        )
        schema.add_field(field_name="pk", datatype=DataType.VARCHAR, max_length=64, is_primary=True)
        schema.add_field(field_name="metadata", datatype=DataType.JSON)
        schema.add_field(field_name="documents", datatype=DataType.VARCHAR, max_length=512)
        schema.add_field(field_name="sparse", datatype=DataType.SPARSE_FLOAT_VECTOR)
        schema.add_field(field_name="embedding", datatype=DataType.FLOAT_VECTOR, dim=vec_dim)

        collection = self.client.create_collection(collection_name=name,
                                       schema=schema,
                                       index_params=index_params)
        self.current_collection = collection
        return collection


    def get_collection(self, name):
        if not self.client.has_collection(name):
            raise ValueError(f"Collection '{name}' does not exist.")
        self.current_collection = Collection(name)
        return self.current_collection

    def get_all_records(self, collection):
        all_records = self.client.query(
            collection_name=collection,
            filter=None,
            output_fields=["documents", "metadata"],
            limit=10000
        )
        if all_records == None:
            all_records = []

        return all_records

    def has_entities(self, name):
        if not self.client.has_collection(name):
            raise ValueError(f"Collection '{name}' does not exists.")
        self.default_collection = name
        collection_stats = self.client.get_collection_stats(collection_name)
        count = collection_stats.get("row_count", 0)  # Retrieve the number of entities
        return count

    def insert(self, collection_name: str, metadata: list[dict[str, any]],
                documents: list[str], sparse_embs: np.ndarray, embeddings: np.ndarray, ids: list[int]):

        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist. Create it first.")

        if len(metadata) != len(embeddings) != len(documents) != len(ids):
           raise ValueError("Metadata, documnets, ids and embeddings must have the same length.")

        data = []
        for meta, doc, sp_embs, emb, id in zip(metadata, documents, sparse_embs, embeddings, ids):
          datum = {
              "pk": id,
              "metadata": meta,
              "documents": doc,
              "sparse": sp_embs,
              "embedding": emb.tolist(),
          }
          data.append(datum)

        self.client.insert(collection_name, data)
        print(f"Inserted {len(metadata)} records into collection '{collection_name}'.")

    def drop_collection(self, collection_name: str):
        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist.")
        self.client.drop_collection(collection_name)
        print(f"Dropped collection '{collection_name}'.")

    def delete_all(self, collection_name: str):
        if not self.client.has_collection(collection_name):
            raise ValueError(f"Collection '{collection_name}' does not exist.")
        self.client.delete(collection_name, expr="pk >= 0")
        self.client.flush([collection_name])

    def search(self, query_embedding: np.ndarray, top_k: int = 10) -> list[dict[str, any]]:
        """
        Search across all collections for the top-k closest embeddings.
        :param query_embedding: The embedding vector to search for.
        :param top_k: Number of top results to retrieve.
        :return: A list of dictionaries containing collection name, id, metadata, and distance.
        """
        results = []
        collections = self.client.list_collections()
        # collections = ["ragbench_collection_techqa_v09"]
        start_time = time.time()
        for collection_name in collections:
            if not self.client.has_collection(collection_name):
                continue

            # Set params to COSINE to match chromadb
            search_params = {"metric_type": "COSINE", "params": {"ef": 128}}

            search_results = self.client.search(
                collection_name=collection_name,
                data=[query_embedding],
                anns_field="embedding",
                search_params=search_params,
                limit=top_k,
                output_fields=["metadata", "documents"]
            )

            print(f"search results size : {len(search_results)}")

            for hits in search_results:
                for hit in hits:
                    print(f"Collection: {collection_name}, data: {str(hit)}")
                    results.append({
                        "collection": collection_name,
                        "id": hit["id"],
                        "metadata": hit["entity"]["metadata"],
                        "distance": hit["distance"],
                        "documents": hit["entity"]["documents"]
                      })

        results = sorted(results, key=lambda x: x["distance"])[:top_k]
        end_time = time.time()
        print(f"Search completed. Found {len(results)} results. in {end_time - start_time} secs")
        return results

    def sparse_search(self, query_embedding: np.ndarray, top_k : int=10)-> list[dict[str, any]] :
        results = []
        collections = self.client.list_collections()
        start_time = time.time()
        for collection_name in collections:
            if not self.client.has_collection(collection_name):
                continue

            # Set params to COSINE to match chromadb
            search_params = {"metric_type": "IP", "params": {"ef": 128}}

            search_results = self.client.search(
                collection_name=collection_name,
                data=[query_embedding],
                anns_field="sparse",
                search_params=search_params,
                limit=top_k,
                output_fields=["metadata", "documents"]
            )

            print(f"search results size : {len(search_results)}")

            for hits in search_results:
                for hit in hits:
                    print(f"Collection: {collection_name}, data: {str(hit)}")
                    results.append({
                        "collection": collection_name,
                        "id": hit["id"],
                        "metadata": hit["entity"]["metadata"],
                        "distance": hit["distance"],
                        "documents": hit["entity"]["documents"]
                      })

        results = sorted(results, key=lambda x: x["distance"])[:top_k]
        end_time = time.time()
        print(f"Search completed. Found {len(results)} results. in {end_time - start_time} secs")
        return results

    def hybrid_search(self, sparse_query_embedding: np.ndarray, dense_query_embedding: np.ndarray, top_k : int=10, alpha=0.3)-> list[dict[str, any]] :
        results = []
        collections = self.client.list_collections()
        start_time = time.time()
        sparse_results = self.sparse_search(sparse_query_embedding, top_k)
        n = int(len(sparse_results) * alpha)
        alpha_sparse_results = sparse_results[:n]
        dense_results = self.search(dense_query_embedding, top_k)
        #'results = sorted(results, key=lambda x: x["distance"])[:top_k]
        results = dense_results + alpha_sparse_results
        end_time = time.time()
        print(f"Hybrid Search completed. Found {len(results)} results. in {end_time - start_time} secs")
        return results

    def extract_documents(self, search_results: list[dict[str, any]]) -> list[np.ndarray]:
      """
      Extract embedding values from search results.
      :param search_results: List of dictionaries containing search results.
      :return: List of embedding vectors as NumPy arrays.
      """
      return [result["documents"] for result in search_results if "documents" in result]

## **Instantiate Milvus and add data to milvus db**

In [13]:
collection_name = "ragbench_collection_techqa_v02"

In [14]:
from sentence_transformers import SentenceTransformer
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained(embedding_model)
embedder = SentenceTransformer(embedding_model)  # Pretrained sentence transformer

In [15]:
datastor = VectorDataStore()

insert_data = False
store_client = "Milvus"
num_records = 0

vector_dim = embedder.get_sentence_embedding_dimension()

datastor.get_or_create_collection(collection_name, vector_dim)
num_records = datastor.has_entities(collection_name)
if num_records == 0:
    insert_data = True

print(f"count >>> {num_records} insert_data >>> {insert_data}")

Collection ragbench_collection_techqa_v02 doesn't exist. Creating...
count >>> 0 insert_data >>> True


# **Store Embeddings into a chroma DB and Milvus**

In [ ]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 459.8/459.8 kB 34.7 MB/s eta 0:00

In [ ]:
import chromadb

client = chromadb.PersistentClient(path="./content/rag_chroma_db_d4")

# chromba_db_collection_name = "ragbench_chroma_db_collection_techqa_v21"
chroma_db_collection = client.get_or_create_collection(name=collection_name)

count = chroma_db_collection.count()
print("count >>>", count)
store_client = "Chromadb"

if count > 0:
    insert_data = False
else:
    insert_data = True


count >>> 0


## **Insert data into database**
### the store_client value stores if the code should call milvus or chromadb code
#### if insert_data is set to true it means there is no data in the collection


In [ ]:
# Adding data to ChromaDB with enhanced duplicate check
existing_hashes = get_existing_hashes(chroma_db_collection)

for i in tqdm(range(0, len(all_documents), batch_size), desc="Adding data to DB"):
    batch_embeddings = all_embeddings[i:i + batch_size]
    batch_metadatas = all_metadatas[i:i + batch_size]
    batch_documents = all_documents[i:i + batch_size]
    batch_ids = []

    # Generate hashes for each document in the batch
    for doc, metadata in zip(batch_documents, batch_metadatas):
        doc_hash = generate_hash(doc, metadata)
        if doc_hash not in existing_hashes:
            batch_ids.append(doc_hash)
            existing_hashes.add(doc_hash)  # Add hash to local set to avoid duplicates in the same batch
        else:
            print(f"Skipping duplicate document: {doc[:50]}...")  # Print a preview of the duplicate doc

    # Add non-duplicate documents to the database
    if batch_ids:  # Ensure there are non-duplicate documents to add
        print(f"Adding {len(batch_ids)} documents to the database... {store_client} with {insert_data}")
        if store_client == "Chromadb" and insert_data:
            chroma_db_collection.add(
                embeddings=batch_embeddings[:len(batch_ids)],  # Trim embeddings to match batch_ids
                metadatas=batch_metadatas[:len(batch_ids)],    # Trim metadatas to match batch_ids
                documents=batch_documents[:len(batch_ids)],    # Trim documents to match batch_ids
                ids=batch_ids
            )


NameError: name 'chroma_db_collection' is not defined

## **Hybrid search implementation** ##
### Sparse vector generation ##

In [16]:
from rank_bm25 import BM25Okapi
from nltk.tokenize import word_tokenize
import scipy.sparse as sp

# Tokenize corpus and prepare BM25 encoder
def prepare_bm25_encoder(texts):
    tokenized_corpus = [word_tokenize(text.lower()) for text in texts]
    bm25_encoder = BM25Okapi(tokenized_corpus)
    return bm25_encoder

def generate_sparse_vector_bm25(query, bm25_encoder):
    tokenized_query = word_tokenize(query.lower())
    scores = bm25_encoder.get_scores(tokenized_query)
    # Convert scores to CSR format
    sparse_vector = sp.csr_matrix(scores)
    return sparse_vector

In [17]:
bm25_encoder = prepare_bm25_encoder(all_documents)

sparse_vectors = [generate_sparse_vector_bm25(text, bm25_encoder) for text in all_documents]

In [18]:
len(sparse_vectors)

128

# **Insert data to milvus**

In [19]:
store_client = "Milvus"

In [20]:
# Adding data to milvus with enhanced duplicate check
all_recs = datastor.get_all_records(collection_name)
#print(f"sample: {str(all_recs[0])}")
existing_hashes = get_existing_hashes_milvus(all_recs)

for i in tqdm(range(0, len(all_documents), batch_size), desc="Adding data to DB"):
    batch_embeddings = all_embeddings[i:i + batch_size]
    batch_sparse_embs = sparse_vectors[i:i + batch_size]
    batch_metadatas = all_metadatas[i:i + batch_size]
    batch_documents = all_documents[i:i + batch_size]
    batch_ids = []

    # Generate hashes for each document in the batch
    for doc, metadata in zip(batch_documents, batch_metadatas):
        doc_hash = generate_hash(doc, metadata)
        if doc_hash not in existing_hashes:
            batch_ids.append(doc_hash)
            existing_hashes.add(doc_hash)  # Add hash to local set to avoid duplicates in the same batch
        else:
            print(f"Skipping duplicate document: {doc[:15]}...")  # Print a preview of the duplicate doc

    # Add non-duplicate documents to the database
    if batch_ids:  # Ensure there are non-duplicate documents to add
        # Add the batch to the Milvus collection
        if store_client == "Milvus" and insert_data:
            datastor.insert(collection_name,
                metadata=batch_metadatas,
                documents=batch_documents,
                sparse_embs = np.array(batch_sparse_embs),
                embeddings=np.array(batch_embeddings),
                ids=batch_ids
            )



print(f"total data inserted into {collection_name} : {datastor.has_entities(collection_name)}")

all records >>> 0


Adding data to DB: 100%|██████████| 1/1 [00:02<00:00,  2.22s/it]

Inserted 128 records into collection 'ragbench_collection_techqa_v02'.
total data inserted into ragbench_collection_techqa_v02 : 128


## **Verifying retrival logic for the Relevant documents**

In [ ]:
# Retrieve from Chroma DB
import time

question ="Why does the other instance of my multi-instance qmgr seem to hang after a failover? Queue manager will not start after failover"
query_embedding = embedder.encode(question).tolist()

# Search for relevant chunks in the vector database
# Retrieve from ChromaDB
start_time = time.time()
results_chroma_db = chroma_db_collection.query(query_embeddings=[query_embedding], n_results=5)
end_time = time.time()
print(f"ChromaDB - Search completed. Found {len(results_chroma_db)} results. in {end_time - start_time} secs")

for idx, (doc, doc_id, metadata) in enumerate(zip(results_chroma_db["documents"][0], results_chroma_db["ids"][0], results_chroma_db["metadatas"][0])):
    print(f"Result {idx + 1}:")
    print(f"  Document: {doc}")
    print(f"  ID: {doc_id}")
    print(f"  Metadata: {metadata}")
    print()


NameError: name 'chroma_db_collection' is not defined

***Desnse Search from Milvus***

In [21]:
# Dense search, Retrive from milvus
import time

question ="Why does the other instance of my multi-instance qmgr seem to hang after a failover? Queue manager will not start after failover"
query_embedding = embedder.encode(question).tolist()

# Retrieve from Milvus
start_time = time.time()
results_milvus = datastor.search(query_embedding, top_k=5)
end_time = time.time()
print(f"Search completed. Found {len(results_milvus)} results. in {end_time - start_time} secs")

for doc in results_milvus:
    print("Relevant Docs from Milvus:\n", doc['documents'])

search results size : 1
Collection: ragbench_collection_techqa_v02, data: {'id': '3683bbefd68a3161c00c001f3e58b2d2', 'distance': 0.8853264451026917, 'entity': {'documents': '[0b] Your WebSphere MQ queue manager fails to come up on the secondary node, and errors are generated.', 'metadata': {'dataset': 'techqa', 'global_index': 0, 'section_index': 0, 'item_index': 1, 'prefix': '0b', 'type': 'Passage'}}}
Collection: ragbench_collection_techqa_v02, data: {'id': '5bb2a66f88c5070dd0f5d37785e3a748', 'distance': 0.8725427389144897, 'entity': {'documents': '[0d] RESOLVING THE PROBLEM\nRename the file amqalchk.fil, which is found under mq\\qmgrs\\qmgrname\\ on the shared drive (to something like amqalchk.fil_OLD); then restart the queue manager.', 'metadata': {'dataset': 'techqa', 'global_index': 0, 'section_index': 0, 'item_index': 3, 'prefix': '0d', 'type': 'Passage'}}}
Collection: ragbench_collection_techqa_v02, data: {'id': 'c2b5c816355deb079e4290bd60b23860', 'distance': 0.8629571199417114,

***Sparse Search from Milvus***

In [22]:
import time

question ="Why does the other instance of my multi-instance qmgr seem to hang after a failover? Queue manager will not start after failover"
query_embedding = generate_sparse_vector_bm25(question, bm25_encoder)

# Retrieve from Milvus
start_time = time.time()
results_milvus = datastor.sparse_search(query_embedding, top_k=5)
end_time = time.time()
print(f"Search completed. Found {len(results_milvus)} results. in {end_time - start_time} secs")

for doc in results_milvus:
    print("Relevant Docs from Milvus:\n", doc['documents'])

search results size : 1
Collection: ragbench_collection_techqa_v02, data: {'id': '8eb8cb34cf1ab14206438ad4f24b28ba', 'distance': 23216.619140625, 'entity': {'documents': '[1j] PROBLEM SUMMARY\n *  ****************************************************************\n   * USERS AFFECTED:                                              *\n   * Users with an HDR pair that is an ER participant             *\n   ****************************************************************\n   * PROBLEM DESCRIPTION:                                         *\n   * After failover, the primary shows online mode but the        *\n   * clients are not able to connect to the server.', 'metadata': {'dataset': 'techqa', 'global_index': 0, 'section_index': 1, 'item_index': 9, 'prefix': '1j', 'type': 'Passage'}}}
Collection: ragbench_collection_techqa_v02, data: {'id': '546a22c406f70dd84821d957db580084', 'distance': 16158.3515625, 'entity': {'documents': '[1l] *\n   *******************************************************

**Hybrid Search from Milvus**

In [23]:
question ="Why does the other instance of my multi-instance qmgr seem to hang after a failover? Queue manager will not start after failover"
# Generate the dense embedding for the question
dense_query_embedding = embedder.encode(question).tolist()

# Generate the sparse embedding for the question
sparse_query_embedding = generate_sparse_vector_bm25(question, bm25_encoder)

# Retrieve from Milvus, passing both dense and sparse embeddings

# Retrieve from Milvus
start_time = time.time()
results_milvus = datastor.hybrid_search(sparse_query_embedding=sparse_query_embedding, dense_query_embedding=dense_query_embedding, top_k=5)
end_time = time.time()
print(f"Hybrid Search completed. Found {len(results_milvus)} results. in {end_time - start_time} secs")

for doc in results_milvus:
    print("Relevant Docs from Milvus:\n", doc['documents'])

search results size : 1
Collection: ragbench_collection_techqa_v02, data: {'id': '8eb8cb34cf1ab14206438ad4f24b28ba', 'distance': 23216.619140625, 'entity': {'documents': '[1j] PROBLEM SUMMARY\n *  ****************************************************************\n   * USERS AFFECTED:                                              *\n   * Users with an HDR pair that is an ER participant             *\n   ****************************************************************\n   * PROBLEM DESCRIPTION:                                         *\n   * After failover, the primary shows online mode but the        *\n   * clients are not able to connect to the server.', 'metadata': {'dataset': 'techqa', 'global_index': 0, 'section_index': 1, 'item_index': 9, 'prefix': '1j', 'type': 'Passage'}}}
Collection: ragbench_collection_techqa_v02, data: {'id': '546a22c406f70dd84821d957db580084', 'distance': 16158.3515625, 'entity': {'documents': '[1l] *\n   *******************************************************

## **Retrival of Relevant Chunks**

In [24]:
# Function to retrieve relevant chunks
def retrieve_docs_milvus(query, top_k=10):
    # Generate embedding for the query
    # Generate the dense embedding for the question
    start_time = time.time()
    dense_query_embedding = embedder.encode(question).tolist()

    # Generate the sparse embedding for the question
    #sparse_query_embedding = generate_sparse_vector_bm25(question, bm25_encoder)

    # Perform vector search to find relevant chunks
    #results = datastor.extract_documents(datastor.search(query_embedding, top_k))
    results = datastor.search( dense_query_embedding, top_k)
    #results = datastor.hybrid_search(sparse_query_embedding, dense_query_embedding, top_k)
    print(f"results: retrieve_docs_milvus >>>  {results}")

    # HyDE search with pseudo document
    #pseudo_docs = fetch_docs_pseudo(query)
    #print(f"pseudo_docs: retrieve_docs_milvus >>>  {pseudo_docs}")

    # Extract 'documents' field
    documents_list = [item['documents'] for item in results]
    #documents_list += pseudo_docs

    end_time = time.time()
    print(f"Hybrid Search completed. Found {len(results)} results (approx. 2 * {top_k} + alpha * {top_k}). in {end_time - start_time} secs")
    # Print the extracted documents
    print("retrieve_docs_milvus >>> documents_list from Hybrid + HyDE search >>>>", documents_list)

    # Extract the retrieved chunks
    # chunks = documents_list
    # should sort and push context - but later

    return documents_list

In [25]:
# Function to retrieve relevant chunks
def retrieve_docs(query, top_k=5):
    # Generate embedding for the query
    query_embedding = embedder.encode(query).tolist()
    # Perform vector search to find relevant chunks
    results = chroma_db_collection.query(query_embeddings=[query_embedding], n_results=top_k)
    # Extract the retrieved chunks
    chunks = results["documents"]
    # should sort and push context - but later
    return chunks

In [26]:
def retrieve_docs_query(query, top_k=5):
    query_embedding = embedder.encode(query)

    if store_client == 'Milvus':
        results = datastor.search(query_embedding, top_k)
        results = datastor.extract_documents(results)
    elif store_client == "Chromadb":
        results = chroma_db_collection.query(query_embeddings=[query_embedding.tolist()], n_results=top_k)
        results = results['documents']
    return results

## **Query Classification**

In [ ]:
!pip install transformers datasets torch

In [ ]:
# Get the databricks dolly dataset (as mentioned in  "Searching for Best Practices in Retrieval-Augmented Generation" paper) for training the query classifer.
from datasets import load_dataset

ds = load_dataset("databricks/databricks-dolly-15k")

README.md:   0%|          | 0.00/8.20k [00:00<?, ?B/s]

databricks-dolly-15k.jsonl:   0%|          | 0.00/13.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# Step 1: Load Pre-trained BERT and Tokenizer
model_name = "bert-base-multilingual-cased"
#model_name = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Step 2: Prepare the Dataset
# Replace with your labeled data
# Example data: {"query": ["What is COVID19?", "What do the S1 and S2 subunits contain?"], "label": [0, 1]}

data = {
    "query": [
        "What is SQLCODE=-1585",
        "The configuration task database-transfer failed with DB2 SQL Error: SQLCODE=-1585, SQLSTATE=54048 While attempting to run the database-transfer task the following error is logged to the ConfigTrace.log: action-process-constraints: Fri Oct 10 13:20:34 CDT 2014 Target started: action-process-constraints [java] Executing java with empty input string [java] [10/10/14 13:20:35.877 CDT] Attempting to create a new Instance of com.ibm.db2.jcc.DB2Driver [java] [10/10/14 13:20:36.016 CDT] Instance of com.ibm.db2.jcc.DB2Driver created successfully [java] [10/10/14 13:20:36.016 CDT] Attempting to make connection using: jdbc:db2://:60500/:returnAlias=0; :: d2svc :: PASSWORD_REMOVED [java] [10/10/14 13:20:36.954 CDT] Connection successfully made [java] [10/10/14 13:20:37.073 CDT] ERROR: Error occurred gathering data from the source database [java] com.ibm.db2.jcc.am.SqlException: DB2 SQL Error: SQLCODE=-1585, SQLSTATE=54048, SQLERRMC=null, DRIVER=4.18.60 [java] at com.ibm.db2.jcc.am.kd.a(kd.java:752)",
        "What is Websphere Application Server",
        "How do I change from shared to unshared connection? in WAS, how do I change from shared to unshared connection. I am seeing connections max out and take a long time to release.",
        "What is JMS",
        "Why is my MQ Java / JMS application getting 2035 NOT_AUTHORIZED error after upgrade of MQ? Why is my MQ Java / JMS application getting 2035 NOT_AUTHORIZED error after upgrade of MQ?",
        "What is TLS",
        "TLS protocol with ITCAM for Datapower We have a DataPower appliance with TLS security protocol enabled. Can we configure ITCAM for DataPower appliance v7.1 to specifically use the TLS protocol v1.2 (not v1.0)?"

    ],
    "label": [0, 1, 0, 1, 0, 1, 0, 1]  # 0: retrieval not needed, 1: retrieval needed
}

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict(data)

#dataset = ds
# Tokenize the Dataset
def preprocess(data):
    return tokenizer(data["query"], padding="max_length", truncation=True, max_length=128)

encoded_dataset = dataset.map(preprocess, batched=True)
encoded_dataset = encoded_dataset.train_test_split(test_size=0.2)

# Step 3: Define the Training Loop
training_args = TrainingArguments(
    output_dir="./query_classifier_results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

# Custom Metric for Accuracy
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    accuracy = (predictions == torch.tensor(labels)).float().mean().item()
    return {"accuracy": accuracy}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Step 4: Train the Model
trainer.train()

# Step 5: Save the Model
model.save_pretrained("./query_classifier")
tokenizer.save_pretrained("./query_classifier")

# Step 6: Inference Function
def classify_query(query, model_path="./query_classifier"):
    # Load saved model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSequenceClassification.from_pretrained(model_path)

    # Tokenize the input query
    inputs = tokenizer(query, return_tensors="pt", padding="max_length", truncation=True, max_length=128)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1).item()

    return predicted_class


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/8 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-29-bd7f59757abd>:66: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.660873,0.500000
2,No log,0.629771,0.500000
3,No log,0.597554,1.000000
4,No log,0.566563,1.000000
5,No log,0.537317,1.000000
6,No log,0.510272,1.000000
7,No log,0.487810,1.000000
8,No log,0.469979,1.000000
9,No log,0.458428,1.000000
10,0.560600,0.452327,1.000000


**Install groq**

In [ ]:
! pip install groq
! pip install -q langchain langchain-groq

**Use LLM for classifying if a query needs RAG or not (instead of using the above classifer)**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

def query_llm_classification(query: str, data: dict):
    """
    Classify whether a query requires retrieval using a few-shot learning approach with Groq.
    """

    # Groq Chat configuration
    chat = ChatGroq(
        temperature=0.3,
        groq_api_key="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",
        model_name="llama3-8b-8192"
    )

    # Few-shot prompt construction
    prompt = ChatPromptTemplate.from_template(
        """
        Decide if the input query needs retrieval for additional context.
        Follow the examples below:

        {few_shot_examples}

        Query: {query}
        Decision:
        """
    )

    # Construct few-shot examples from the provided data
    few_shot_examples = ""
    for q, label in zip(data["query"], data["label"]):
        decision = "retrieval required" if label == 1 else "retrieval not required"
        few_shot_examples += f"Query: {q}\nDecision: {decision}\n\n"

    # Format the input to the Groq model
    input_data = {
        "few_shot_examples": few_shot_examples.strip(),
        "query": query
    }

    # Generate the response using the chain
    chain = prompt | chat
    groq_response = chain.invoke(input_data)

    # Extract and return the decision
    decision = groq_response+"".strip()
    return decision

data = {
    "query": [
        "What is SQLCODE=-1585",
        "The configuration task database-transfer failed with DB2 SQL Error: SQLCODE=-1585...",
        "What is Websphere Application Server",
        "How do I change from shared to unshared connection?",
        "What is JMS",
        "Why is my MQ Java / JMS application getting 2035 NOT_AUTHORIZED error after upgrade?",
        "What is TLS",
        "TLS protocol with ITCAM for Datapower..."
    ],
    "label": [0, 1, 0, 1, 0, 1, 0, 1]
}

#new_query = "How can I resolve SQLCODE=-1585 in DB2?"
new_query = "What is Java Messaging Service?"
decision = query_llm_classification(new_query, data)

print(f"Query: {new_query}\nDecision: {decision}")


Query: What is Java Messaging Service?
Decision: input_variables=[] input_types={} partial_variables={} messages=[AIMessage(content='Decision: retrieval not required', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 187, 'total_tokens': 193, 'completion_time': 0.005, 'prompt_time': 0.033584515, 'queue_time': 0.01802098, 'total_time': 0.038584515}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None}, id='run-fb3873f0-e0dc-48a0-843e-9ab58dc647c2-0', usage_metadata={'input_tokens': 187, 'output_tokens': 6, 'total_tokens': 193}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template=''), additional_kwargs={})]


# **Use cross encoder for re-ranking**

In [27]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

# Load Cross-Encoder model and tokenizer
cross_encoder_model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2"
tokenizer = AutoTokenizer.from_pretrained(cross_encoder_model_name)
model = AutoModelForSequenceClassification.from_pretrained(cross_encoder_model_name)

# Ensure PyTorch is in evaluation mode
model.eval()

def rerank_with_cross_encoder(query):
    """
    Rerank documents based on relevance scores from a Cross-Encoder model.

    Args:
        query (str): The query string.
        documents (list): A list of document strings.

    Returns:
        list: A list of tuples (document, score), sorted by score in descending order.
    """
    query_embedding = embedder.encode(question)
    results = datastor.search(query_embedding, 5)
    documents = datastor.extract_documents(results)

    scores = []
    for doc in documents:
        # Tokenize query-document pair
        inputs = tokenizer(
            query,
            doc,
            return_tensors="pt",
            max_length=512,  # Limit for most transformer models
            truncation=True,
            padding="max_length",
        )
        # Compute relevance scores
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits

            # Handle binary classification or regression logits
            if logits.size(1) == 2:  # Binary classification
                score = torch.softmax(logits, dim=1)[:, 1].item()  # Probability of relevance (class 1)
            else:  # Regression or single-class output
                score = logits.squeeze().item()  # Direct score (e.g., relevance regression)

            scores.append((doc, score))
    # Sort by score in descending order
    return sorted(scores, key=lambda x: x[1], reverse=True)



tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

## **Presort of results with monoT5 model** ##

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, T5Tokenizer, T5ForConditionalGeneration
import torch.nn.functional as F
import torch

# Load MonoT5 model and tokenizer
monot5_model_name = "castorini/monot5-base-msmarco"  # or a suitable MonoT5 variant
#monot5_tokenizer = AutoTokenizer.from_pretrained(monot5_model_name)
#monot5_model = AutoModelForSeq2SeqLM.from_pretrained(monot5_model_name)

monot5_tokenizer = T5Tokenizer.from_pretrained(monot5_model_name)
monot5_model = T5ForConditionalGeneration.from_pretrained(monot5_model_name)


tokenizer_config.json:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [ ]:
pip install python-terrier

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.4/163.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.9/347.9 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.3 MB/s eta 0:00:00
  Created wheel for chest: filename=chest-0.2.3-py3-none-any.whl size=7611 sha256=09190c68906629b46dca9227d0bc0b5268130c999a69ca8e8341c112be323935
  Stored in directory: /root/.cache/pip/wheels/48/57/13/

In [ ]:
pip install pyterrier_t5

In [ ]:
import pyterrier as pt
from pyterrier_t5 import MonoT5ReRanker
# Initialize PyTerrier (if not already initialized)
if not pt.started():
    pt.init()

def transform_to_tuples(doc_list):
    transformed = []
    for doc in doc_list:
        transformed.append((doc,))  # Wrap each document in a tuple
    return transformed


def rerank_with_monot5(question):
  # Initialize MonoT5 reranker
  monot5 = MonoT5ReRanker()

  query_embedding = embedder.encode(question)

  db_docs = datastor.search(query_embedding, 5)

  df_docs = pd.DataFrame({"text": db_docs})
  df_docs["qid"] = 1  # Query ID
  df_docs["query"] = question  # Add query column

  # Apply MonoT5 reranking
  reranked_docs = monot5.transform(df_docs)

  # Sort documents by rank (ascending)
  reranked_docs_sorted = reranked_docs.sort_values(by="score", ascending=False)
  print("reranked_docs_sorted>>>",reranked_docs_sorted)
  monot5_doc_list=[]
  # Print reranked and sorted documents
  for _, doc in reranked_docs_sorted.iterrows():
      print(f"Rank: {doc['score']}, Doc: {doc['text']}")
      monot5_doc_list.append(doc['text']['documents'])
  monot5_doc_list = transform_to_tuples(monot5_doc_list)
  return monot5_doc_list

<ipython-input-38-ab62883d41d1>:4: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


 ### function to sort data ###   

In [ ]:
def sort_documents(query, documents, top_n=5):

  # Prepare inputs
  inputs = [f"Query: {query} Document: {doc}" for doc in documents]

  # Tokenize inputs
  tokenized_inputs = monot5_tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")

  # Get model predictions
  outputs = monot5_model.generate(**tokenized_inputs)

  # Decode outputs to get relevance scores (e.g., 'true' or 'false')
  predictions = [monot5_tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

  # Convert predictions to scores
  scores = [1.0 if pred.lower() == "true" else 0.0 for pred in predictions]

  # Rank documents
  ranked_docs = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)

  # Output ranked documents
  results = []
  for i, (doc, score) in enumerate(ranked_docs, start=1):
    print(f"debug: Rank {i}: {doc} (Score: {score})")
    if score == 1.0:
      if isinstance(doc, str):
        results.append(doc)
      else:
        results.append(doc['documents'])

  print(f"debug: results: {results}")

  return results[:top_n]

### Test code ###

In [ ]:
'''
# call sort documents

print(f"quesiton: {question}")
db_docs = datastor.extract_documents(results_milvus)
print(f"documents: {db_docs}")

result_docs = sort_documents(question, db_docs)
print("result_docs >>>",result_docs)
'''
# Test with monot5 re-ranker
monot5_reranked_doc_list = rerank_with_monot5(question)
print("monot5_reranked_doc_list >>> ",monot5_reranked_doc_list)

search results size : 1
Collection: ragbench_collection_techqa_v01, data: {'id': '3683bbefd68a3161c00c001f3e58b2d2', 'distance': 0.8853264451026917, 'entity': {'documents': '[0b] Your WebSphere MQ queue manager fails to come up on the secondary node, and errors are generated.', 'metadata': {'dataset': 'techqa', 'global_index': 0, 'section_index': 0, 'item_index': 1, 'prefix': '0b', 'type': 'Passage'}}}
Collection: ragbench_collection_techqa_v01, data: {'id': '5bb2a66f88c5070dd0f5d37785e3a748', 'distance': 0.8725427389144897, 'entity': {'documents': '[0d] RESOLVING THE PROBLEM\nRename the file amqalchk.fil, which is found under mq\\qmgrs\\qmgrname\\ on the shared drive (to something like amqalchk.fil_OLD); then restart the queue manager.', 'metadata': {'dataset': 'techqa', 'global_index': 0, 'section_index': 0, 'item_index': 3, 'prefix': '0d', 'type': 'Passage'}}}
Collection: ragbench_collection_techqa_v01, data: {'id': 'c2b5c816355deb079e4290bd60b23860', 'distance': 0.8629571199417114,

monoT5:   0%|          | 0/2 [00:00<?, ?batches/s]

reranked_docs_sorted>>>                                                 text  qid  \
4  {'collection': 'ragbench_collection_techqa_v01...    1   
3  {'collection': 'ragbench_collection_techqa_v01...    1   
1  {'collection': 'ragbench_collection_techqa_v01...    1   
0  {'collection': 'ragbench_collection_techqa_v01...    1   
2  {'collection': 'ragbench_collection_techqa_v01...    1   

                                               query     score  rank  
4  Why does the other instance of my multi-instan... -0.060666     0  
3  Why does the other instance of my multi-instan... -0.282272     1  
1  Why does the other instance of my multi-instan... -0.536859     2  
0  Why does the other instance of my multi-instan... -2.973851     3  
2  Why does the other instance of my multi-instan... -4.154908     4  
Rank: -0.06066613271832466, Doc: {'collection': 'ragbench_collection_techqa_v01', 'id': '3683bbefd68a3161c00c001f3e58b2d2', 'metadata': {'dataset': 'techqa', 'global_index': 0, 'secti

In [28]:
# Re-rank the documents with cross encoder

reranked_docs = rerank_with_cross_encoder(question)

# Display results
print("Cross Encoder Re-ranked Documents:")
for rank, (doc, score) in enumerate(reranked_docs, 1):
    print(f"Rank {rank}: Score = {score:.4f}, Doc = {doc}")


search results size : 1
Collection: ragbench_collection_techqa_v02, data: {'id': '3683bbefd68a3161c00c001f3e58b2d2', 'distance': 0.8853264451026917, 'entity': {'documents': '[0b] Your WebSphere MQ queue manager fails to come up on the secondary node, and errors are generated.', 'metadata': {'dataset': 'techqa', 'global_index': 0, 'section_index': 0, 'item_index': 1, 'prefix': '0b', 'type': 'Passage'}}}
Collection: ragbench_collection_techqa_v02, data: {'id': '5bb2a66f88c5070dd0f5d37785e3a748', 'distance': 0.8725427389144897, 'entity': {'documents': '[0d] RESOLVING THE PROBLEM\nRename the file amqalchk.fil, which is found under mq\\qmgrs\\qmgrname\\ on the shared drive (to something like amqalchk.fil_OLD); then restart the queue manager.', 'metadata': {'dataset': 'techqa', 'global_index': 0, 'section_index': 0, 'item_index': 3, 'prefix': '0d', 'type': 'Passage'}}}
Collection: ragbench_collection_techqa_v02, data: {'id': 'c2b5c816355deb079e4290bd60b23860', 'distance': 0.8629571199417114,

# **HyDE Implementation with an LLM to generate a pseudo document** ##

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=0.3, groq_api_key="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", model_name="llama3-8b-8192")

def generate_pseudo_document_llm(query: str):

    prompt = ChatPromptTemplate.from_template(
      """
      Please provide a response to the query below, in a passage.
      Respond just with the passage only.

      Question: {query}

      Passage:
      """
    )

    chain = prompt | chat

    groq_response = chain.invoke({"query": query})

    groq_response.pretty_print()

    answer = groq_response.content
    return answer


def get_pseudo_doc_embeddings(answer: str):
    chunks = chunk_with_token_limit(answer, TOKEN_LIMIT, SLIDING_WINDOW_OVERLAP)
    return embedder.encode(chunks).tolist()

def fetch_docs_pseudo(query: str):
    answer = generate_pseudo_document_llm(query)
    pseudo_doc_embeddings = get_pseudo_doc_embeddings(answer)
    return datastor.extract_documents(datastor.search(pseudo_doc_embeddings))


In [ ]:
question = "When was Rolex registered?"

answer = generate_pseudo_document_llm(question)
print(f'answer {answer}')
doc_results = fetch_docs_pseudo(question)

print(f'doc_results: {doc_results}')

# **Doc Summarization (Recomp)**

In [ ]:
from transformers import pipeline

# Initialize the summarization pipeline
summarizer = pipeline('summarization')

# Abstractive Compression: Generate a concise summary by synthesizing information from multiple documents.
def summarize_docs(retrieved_docs):

    # Concatenate documents and perform abstractive summarization
    concatenated_docs = ' '.join(retrieved_docs)
    summary = summarizer(concatenated_docs, max_length=150, min_length=30, do_sample=False)[0]['summary_text']

    return summary


No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

Device set to use cuda:0


# **LLM Inference with groq**

In [29]:
! pip install langchain
! pip install groq
! pip install -q langchain langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 5.7 MB/s eta 0:00:00


In [30]:
import langchain

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

# Initializing the context variable which later gets populated with retrieved chunks
context = ""

In [31]:
def query_response_from_llm(query: str):

    # retrieve chunks from milvus db
    chunks = retrieve_docs_milvus(query)

    # retrieve chunks from chroma db
    #chunks = retrieve_docs(query)

    # Flatten the list if necessary
    if any(isinstance(chunk, list) for chunk in chunks):
      chunks = [item for sublist in chunks for item in (sublist if isinstance(sublist, list) else [sublist])]

    chat = ChatGroq(temperature=0.3, groq_api_key="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", model_name="llama3-8b-8192")

    #chat = ChatGroq(temperature=0.3, groq_api_key="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", model_name="deepseek-r1-distill-llama-70b")

    prompt=ChatPromptTemplate.from_template(
      """
      Please provide a response to the query below, strictly adhering to the
      information presented in the following documents.
      Do not generate any text beyond what is explicitly stated in the documents.

      Context: {context}

      Question: {query}

      Answer:
      """
    )


    # Summarize the retrieved doc chunks
    # Compress documents using abstractive summarization before appending to context
    chunk_summary = ""

    '''
    print("chunks>>>>",chunks)
    sorted_chunks = sort_documents(query, chunks)
    print(f"sorted_chunks: {sorted_chunks}")
    chunk_summary = summarize_docs(sorted_chunks)
    print("chunk_summary>>>>",chunk_summary)
    chunks = chunk_summary.join(chunks)
    '''

    #Cross encoder reranked docs
    reranked_chunks = rerank_with_cross_encoder(question)

    #monot5 reranked docs
    #reranked_chunks = rerank_with_monot5(question)

    doc_chunks = [chunk[0] for chunk in reranked_chunks]
    print("doc_chunks>>after re-ranking>>", doc_chunks)
    chain = prompt | chat

    context = "".join(doc_chunks)

    print("context>>>from 1st RAG>>>>>> ",context)

    groq_response = chain.invoke({"context": context, "query": query})

    print("groq_response>>>from 1st RAG>>>>>> ",groq_response)

    answer = groq_response
    return answer, context

In [ ]:
def query_response_from_llm_no_rag(query: str):

    chat = ChatGroq(temperature=0.3, groq_api_key="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", model_name="llama3-8b-8192")

    prompt=ChatPromptTemplate.from_template(
      """
      Please provide a response to the query below

      Question: {query}

      Answer:
      """
    )

    chain = prompt | chat

    groq_response_no_rag = chain.invoke({"query": query})

    print("groq_response_no_rag>>>no RAG>>>>>> ",groq_response_no_rag)

    answer = groq_response_no_rag
    return answer

## **USER QUERY**

In [32]:
qna_list = []

In [33]:
#query = "What are the most frequent clinical manifestations of human adenovirus type 55 (HAdV-55) induced ARDS?"
#query ="The configuration task database-transfer failed with DB2 SQL Error: SQLCODE=-1585, SQLSTATE=54048 While attempting to run the database-transfer task the following error is logged to the ConfigTrace.log: action-process-constraints: Fri Oct 10 13:20:34 CDT 2014 Target started: action-process-constraints [java] Executing java with empty input string [java] [10/10/14 13:20:35.877 CDT] Attempting to create a new Instance of com.ibm.db2.jcc.DB2Driver [java] [10/10/14 13:20:36.016 CDT] Instance of com.ibm.db2.jcc.DB2Driver created successfully [java] [10/10/14 13:20:36.016 CDT] Attempting to make connection using: jdbc:db2://:60500/:returnAlias=0; :: d2svc :: PASSWORD_REMOVED [java] [10/10/14 13:20:36.954 CDT] Connection successfully made [java] [10/10/14 13:20:37.073 CDT] ERROR: Error occurred gathering data from the source database [java] com.ibm.db2.jcc.am.SqlException: DB2 SQL Error: SQLCODE=-1585, SQLSTATE=54048, SQLERRMC=null, DRIVER=4.18.60 [java] at com.ibm.db2.jcc.am.kd.a(kd.java:752)"
#question = "Why does the other instance of my multi-instance qmgr seem to hang after a failover? Queue manager will not start after failover."

datasets = ['covidqa','cuad','delucionqa','emanual','expertqa','finqa','hagrid','hotpotqa','msmarco','pubmedqa','tatqa','techqa']

data = load_dataset("rungalileo/ragbench", datasets[11], split="train")
top_5_rows = data.select(range(5))
for i, row in enumerate(top_5_rows):
    print(f"Row {i + 1}:")
    question = row['question']

    # Query Classification
    #classification = classify_query(query)
    classification = 1
    #print('classification >>>> ',classification)

    if classification == 1:
        print("Perform retrieval using RAG.")
        # Call RAG pipeline for retrieval and response generation
        #answer,context = query_response_from_llm(query)
        print('Question for LLM>>>>', question)
        answer,context = query_response_from_llm(question)
    else:
        print("No RAG - only use LLM for response generation.")
        # Call LLM directly
        answer = query_response_from_llm_no_rag(question)

    print("context  >>from llm>>> ", context)
    print("answer  >>from llm>>> ", answer)
    # Store question, answer, and context in the list
    qna_list.append({"question": question, "answer": answer, "context": context})

Row 1:
Perform retrieval using RAG.
Question for LLM>>>> Why does the other instance of my multi-instance qmgr seem to hang after a failover? Queue manager will not start after failover.
search results size : 1
Collection: ragbench_collection_techqa_v02, data: {'id': '3683bbefd68a3161c00c001f3e58b2d2', 'distance': 0.8994988799095154, 'entity': {'documents': '[0b] Your WebSphere MQ queue manager fails to come up on the secondary node, and errors are generated.', 'metadata': {'dataset': 'techqa', 'global_index': 0, 'section_index': 0, 'item_index': 1, 'prefix': '0b', 'type': 'Passage'}}}
Collection: ragbench_collection_techqa_v02, data: {'id': '5bb2a66f88c5070dd0f5d37785e3a748', 'distance': 0.8820456266403198, 'entity': {'documents': '[0d] RESOLVING THE PROBLEM\nRename the file amqalchk.fil, which is found under mq\\qmgrs\\qmgrname\\ on the shared drive (to something like amqalchk.fil_OLD); then restart the queue manager.', 'metadata': {'dataset': 'techqa', 'global_index': 0, 'section_in

## **PROMPT for generating metrics as JSON response**

In [34]:
def generate_prompt():
    """
    Generate a prompt template for assessing the support and relevance of an LLM-generated response.
    """
    return """
    I asked someone to answer a question based on one or more documents.
    Your task is to review their response and assess whether or not each sentence
    in that response is supported by text in the documents. And if so, which
    sentences in the documents provide that support. You will also tell me which
    of the documents contain useful information for answering the question, and
    which of the documents the answer was sourced from.
    Here are the documents, each of which is split into sentences.Alongside each
    sentence is associated key, such as ’[0a].’ or ’[0b].’ that you can use to refer
    to it:

    ‘‘‘
    {documents}
    ‘‘‘
    The question was:
    ‘‘‘
    {question}
    ‘‘‘

    Here is their response, split into sentences. Alongside each sentence is
    associated key, such as ’a.’ or ’b.’ that you can use to refer to it. Note
    that these keys are unique to the response, and are not related to the keys
    in the documents:
    ‘‘‘
    {answer}
    ‘‘‘
    You must respond with a JSON object matching this schema:
    ‘‘‘
    {{
    "relevance_explanation": string,
    "all_relevant_sentence_keys": [string],
    "overall_supported_explanation": string,
    "overall_supported": boolean,
    "sentence_support_information": [
    {{
    "response_sentence_key": string,
    "explanation": string,
    "supporting_sentence_keys": [string],
    "fully_supported": boolean
    }},
    ],
    "all_utilized_sentence_keys": [string]
    }}
    ‘‘‘
    The relevance_explanation field is a string explaining which documents
    contain useful information for answering the question. Provide a step-by-step
    breakdown of information provided in the documents and how it is useful for
    answering the question.
    The all_relevant_sentence_keys field is a list of all document sentences keys
    (e.g. ’0a’) that are relevant to the question. Include every sentence that is
    useful and relevant to the question, even if it was not used in the response,
    or if only parts of the sentence are useful. Ignore the provided response when
    making this judgement and base your judgement solely on the provided documents
    and question. Omit sentences that, if removed from the document, would not
    impact someone’s ability to answer the question.
    The overall_supported_explanation field is a string explaining why the response
    *as a whole* is or is not supported by the documents. In this field, provide a
    step-by-step breakdown of the claims made in the response and the support (or
    lack thereof) for those claims in the documents. Begin by assessing each claim
    separately, one by one; don’t make any remarks about the response as a whole
    until you have assessed all the claims in isolation.
    The overall_supported field is a boolean indicating whether the response as a
    whole is supported by the documents. This value should reflect the conclusion
    you drew at the end of your step-by-step breakdown in overall_supported_explanation.
    In the sentence_support_information field, provide information about the support
    *for each sentence* in the response.
    The sentence_support_information field is a list of objects, one for each sentence
    in the response. Each object MUST have the following fields:
    - response_sentence_key: a string identifying the sentence in the response.
    This key is the same as the one used in the response above.

    - explanation: a string explaining why the sentence is or is not supported by the
    documents.
    - supporting_sentence_keys: keys (e.g. ’[0a]’) of sentences from the documents that
    support the response sentence. If the sentence is not supported, this list MUST
    be empty. If the sentence is supported, this list MUST contain one or more keys.
    In special cases where the sentence is supported, but not by any specific sentence,
    you can use the string "supported_without_sentence" to indicate that the sentence
    is generally supported by the documents. Consider cases where the sentence is
    expressing inability to answer the question due to lack of relevant information in
    the provided context as "supported_without_sentence". In cases where the sentence
    is making a general statement (e.g. outlining the steps to produce an answer, or
    summarizing previously stated sentences, or a transition sentence), use the
    string "general". In cases where the sentence is correctly stating a well-known fact,
    like a mathematical formula, use the string "well_known_fact". In cases where the
    sentence is performing numerical reasoning (e.g. addition, multiplication), use
    the string "numerical_reasoning".
    - fully_supported: a boolean indicating whether the sentence is fully supported by
    the documents.
    - This value should reflect the conclusion you drew at the end of your step-by-step
    breakdown in explanation.
    - If supporting_sentence_keys is an empty list, then fully_supported must be false.
    - Otherwise, use fully_supported to clarify whether everything in the response
    sentence is fully supported by the document text indicated in supporting_sentence_keys
    (fully_supported = true), or whether the sentence is only partially or incompletely
    supported by that document text (fully_supported = false).
    The all_utilized_sentence_keys field is a list of all sentences keys (e.g. ’0a’) that
    were used to construct the answer. Include every sentence that either directly supported
    the answer, or was implicitly used to construct the answer, even if it was not used
    in its entirety. Omit sentences that were not used, and could have been removed from
    the documents without affecting the answer.
    You must respond with a valid JSON string. Use escapes for quotes, e.g. \\"\\", and
    newlines, e.g. \\n. Do not write anything before or after the JSON string. Do not
    wrap the JSON string in backticks like ‘‘‘ or ‘‘‘json.
    As a reminder: your task is to review the response and assess which documents contain
    useful information pertaining to the question, and how each sentence in the response
    is supported by the text in the documents.
    """.strip()


## **Response generation - llama3-70b-8192 and deepseek-r1-distill-llama-70b**

In [35]:
qna_with_json_list = []

In [36]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=0.3, groq_api_key="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", model_name="llama3-70b-8192")
#chat = ChatGroq(temperature=0.3, groq_api_key="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", model_name="deepseek-r1-distill-llama-70b")

prompt_template_with_docs = PromptTemplate(
    input_variables=["documents", "question", "answer"],
    template=generate_prompt(),
)

for j, r in enumerate(qna_list):
    print(f"Row {j + 1}:")
    query_for_llm = r['question']
    answer_for_llm = r['answer']
    context_for_llm = r['context']

    print('context for groq >>>> ', context_for_llm)
    print('query for groq >>>> ', query_for_llm)
    print('answer for groq >>>> ', answer_for_llm)

    chain = prompt_template_with_docs | chat

    # if classification == 1:
    #  groq_response_with_context_qanda = chain.invoke({"documents": context, "question": query, "answer":groq_response})
    # else
    #  groq_response_with_out-rag = chain.invoke({"documents": "", "question": query, "answer":groq_response})

    groq_response_with_context_qanda = chain.invoke({"documents": context_for_llm, "question": query_for_llm, "answer":answer_for_llm})

    print("groq_response>>>>with context, query and answer>>>>> ",groq_response_with_context_qanda)
    # Store question and answer in the list
    qna_with_json_list.append({"question": query_for_llm, "answer": groq_response_with_context_qanda})

Row 1:
context for groq >>>>  [0d] RESOLVING THE PROBLEM
Rename the file amqalchk.fil, which is found under mq\qmgrs\qmgrname\ on the shared drive (to something like amqalchk.fil_OLD); then restart the queue manager.[1j] PROBLEM SUMMARY
 *  ****************************************************************
   * USERS AFFECTED:                                              *
   * Users with an HDR pair that is an ER participant             *
   ****************************************************************
   * PROBLEM DESCRIPTION:                                         *
   * After failover, the primary shows online mode but the        *
   * clients are not able to connect to the server.[0b] Your WebSphere MQ queue manager fails to come up on the secondary node, and errors are generated.[0a] HL083112 mqlpgrlg ZX000001 ExecCtrlrMain lpiRC_LOG_NOT_AVAILABLE mscs TECHNOTE (TROUBLESHOOTING)

PROBLEM(ABSTRACT)
 You attempt to failover from the primary to secondary node under MSCS.[1h] If t

## **JSON Data parsing to retrieve metrics**

In [37]:
import re
import json

In [52]:
json_str_list = []

In [53]:
# Extract the content field using regular expressions
for k, ro in enumerate(qna_with_json_list):
    print(f"Row {k + 1}:")
    groq_response_with_context_qanda = ro['answer']
    content_match = re.search(r"content='(.*?)' additional_kwargs=", str(groq_response_with_context_qanda), re.DOTALL)
    if content_match:
        content = content_match.group(1)
        print("Extracted Content:")
        print(content)

        json_match = re.search(r"\{.*\}", content, re.DOTALL)
        if json_match:
          json_str = json_match.group(0)

          json_str = json_str.replace("'", '"').replace("\\n","").replace("\\","")
          print(json_str)
          ques = ro['question']
          try:
            # Parse the JSON
            parsed_json = json.loads(json_str)
            print("Extracted JSON:")
            print(json.dumps(parsed_json, indent=4))
            json_str_list.append({"question":ques, "jsonstr": parsed_json})

          except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
        else:
          print("No JSON found in the provided string.")

    else:
        print("Content field not found in the provided string.")


Row 1:
Extracted Content:
{\n"relevance_explanation": "The document contains useful information for answering the question, specifically the problem description and the solution provided.",\n"all_relevant_sentence_keys": ["[0a]", "[0b]", "[0d]", "[1h]", "[1j]"],\n"overall_supported_explanation": "The response provides a solution to the problem, which is supported by the document. The document describes the problem of the queue manager failing to come up on the secondary node after failover, and the solution provided in the response is to rename the file amqalchk.fil and restart the queue manager, which is supported by sentence [0d].",\n"overall_supported": true,\n"sentence_support_information": [\n{\n"response_sentence_key": "a.",\n"explanation": "The sentence is supported by the document, specifically sentence [0d], which provides the solution to the problem.",\n"supporting_sentence_keys": ["[0d]"],\n"fully_supported": true\n},\n{\n"response_sentence_key": "b.",\n"explanation": "The s

## **Computation Metrics from JSON response in comparison with ground truth**

In [40]:
import json

In [54]:
# Helper function for length computation (mocked as sentence count here)
def compute_length(keys):
    return len(keys)

# Metrics Computation
def compute_metrics(data):

    all_relevant = data["all_relevant_sentence_keys"]
    all_utilized = data["all_utilized_sentence_keys"]
    sentences_info = data["sentence_support_information"]

    # Context Relevance
    total_relevant_length = compute_length(all_relevant)
    total_context_length = total_relevant_length  # Assuming all relevant are part of the context
    context_relevance = total_relevant_length / total_context_length if total_context_length > 0 else 0

    # Context Utilization
    total_utilized_length = compute_length(all_utilized)
    context_utilization = total_utilized_length / total_context_length if total_context_length > 0 else 0

    # Completeness
    total_relevant_utilized = sum(
        1 for s in sentences_info if set(s["supporting_sentence_keys"]).intersection(all_utilized)
    )
    print("total_relevant_utilized>>>",total_relevant_utilized)
    print("total_relevant_length>>>",total_relevant_length)
    completeness = total_relevant_utilized / total_relevant_length if total_relevant_length > 0 else 0


    # Adherence
    adherence = all(s["fully_supported"] for s in sentences_info)

    return {
        "Context Relevance": context_relevance,
        "Context Utilization": context_utilization,
        "Completeness": completeness,
        "Adherence": adherence
    }

In [74]:
predicted_metrics_list =[]

In [76]:
for m, json_row in enumerate(json_str_list):
    data_str = json_row['jsonstr']
    try:
      #data = json.loads(data_str)  # Parse JSON string into a dictionary
      # Compute and print metrics
      predicted_metrics = compute_metrics(data_str)
      print(json.dumps(predicted_metrics, indent=4))
      predicted_metrics_list.append({"question":question,"predicted_metrics":predicted_metrics})
    except json.JSONDecodeError as e:
      print(f"Error decoding JSON at row {m + 1}: {e}")



total_relevant_utilized>>> 1
total_relevant_length>>> 5
{
    "Context Relevance": 1.0,
    "Context Utilization": 0.2,
    "Completeness": 0.2,
    "Adherence": true
}
total_relevant_utilized>>> 1
total_relevant_length>>> 4
{
    "Context Relevance": 1.0,
    "Context Utilization": 0.5,
    "Completeness": 0.25,
    "Adherence": false
}
total_relevant_utilized>>> 1
total_relevant_length>>> 3
{
    "Context Relevance": 1.0,
    "Context Utilization": 0.6666666666666666,
    "Completeness": 0.3333333333333333,
    "Adherence": true
}
total_relevant_utilized>>> 2
total_relevant_length>>> 2
{
    "Context Relevance": 1.0,
    "Context Utilization": 1.0,
    "Completeness": 1.0,
    "Adherence": true
}


## **Fetching the groud truth values**

In [77]:
def fetch_ground_truth(row, question):
    #for sample in dataset["train"]:  # Change "train" to the correct split if needed
    if row["question"] == question:
        return {
            "Context Relevance": row["relevance_score"],  # Adjust column name if needed
            "Context Utilization": row["utilization_score"],  # Adjust column name if needed
            "Adherence": row["adherence_score"]  # Adjust column name if needed
        }
    return None


In [78]:
ground_truth_list=[]

In [79]:
sub_dataset = load_dataset("rungalileo/ragbench", datasets[11], split="train")
top_5_rows = data.select(range(5))
for i, row in enumerate(top_5_rows):
    question = row['question']
    print("question>>>",question)
    ground_truth = fetch_ground_truth(row, question)

    if ground_truth:
        print("Ground Truth Values:")
        print(json.dumps(ground_truth, indent=4))
        ground_truth_list.append({"question":question, "ground_truth": ground_truth})
    else:
        print(f"Question not found in the dataset: {question}")

question>>> Why does the other instance of my multi-instance qmgr seem to hang after a failover? Queue manager will not start after failover.
Ground Truth Values:
{
    "Context Relevance": 0.01411764705882353,
    "Context Utilization": 0.009411764705882352,
    "Adherence": true
}
question>>> DASH 3.1.2.1 to 3.1.3.0 taking long time DASH 3.1.2.1 to 3.1.3.0 taking long time. How to fix this?
Ground Truth Values:
{
    "Context Relevance": 0.0,
    "Context Utilization": 0.0,
    "Adherence": false
}
question>>> Why cant I load and AEL when using IE 11 JRE 8 Application Blocked by Java Security

Ground Truth Values:
{
    "Context Relevance": 0.12631578947368421,
    "Context Utilization": 0.021052631578947368,
    "Adherence": false
}
question>>> Where is latest information on fix packs for Cast Iron? Where can I find more information on the fix packs and downloads available for the various releases of Cast Iron? 
Ground Truth Values:
{
    "Context Relevance": 0.14285714285714285,
  

## **Evaluation Metrics**

In [80]:
from sklearn.metrics import mean_squared_error, roc_auc_score
import numpy as np
import json

# **RMSE**

In [81]:
def compute_rmse(predicted, ground_truth):

    # Extract true and predicted values
    y_true_relevance = ground_truth["Context Relevance"]
    y_true_utilization = ground_truth["Context Utilization"]

    y_pred_relevance = predicted["Context Relevance"]
    y_pred_utilization = predicted["Context Utilization"]

    # Compute RMSE for Context Relevance and Context Utilization
    rmse_relevance = np.sqrt((y_pred_relevance - y_true_relevance) ** 2)
    rmse_utilization = np.sqrt((y_pred_utilization - y_true_utilization) ** 2)

    return {
        "RMSE-Relevance": rmse_relevance,
        "RMSE-Utililization": rmse_utilization,
    }


In [101]:
def compute_rmse_list(predicted_list, ground_truth_list):
    rmse_relevance_list = []
    rmse_utilization_list = []

    # Iterate through the lists of predicted and ground truth values
    for predicted, ground_truth in zip(predicted_list, ground_truth_list):
        # Extract true and predicted values for each pair
        y_true_relevance = ground_truth["ground_truth"]["Context Relevance"]
        y_true_utilization = ground_truth["ground_truth"]["Context Utilization"]

        y_pred_relevance = predicted["predicted_metrics"]["Context Relevance"]
        y_pred_utilization = predicted["predicted_metrics"]["Context Utilization"]

        # Compute RMSE for Context Relevance and Context Utilization
        rmse_relevance = np.sqrt((y_pred_relevance - y_true_relevance) ** 2)
        rmse_utilization = np.sqrt((y_pred_utilization - y_true_utilization) ** 2)

        # Append the results to the lists
        rmse_relevance_list.append(rmse_relevance)
        rmse_utilization_list.append(rmse_utilization)

    return {
        "RMSE-Relevance": rmse_relevance_list,
        "RMSE-Utilization": rmse_utilization_list,
    }



In [102]:
evaluation_metrics_list = compute_rmse_list(predicted_metrics_list, ground_truth_list)
print(evaluation_metrics_list["RMSE-Relevance"])
print(evaluation_metrics_list["RMSE-Utilization"])

[0.9858823529411764, 1.0, 0.8736842105263158, 0.8571428571428572, 0.8853754940711462]
[0.19058823529411767, 0.5, 0.6456140350877193, 0.8660714285714286, 0.13280632411067195]


# **AUCROC**

In [107]:
from sklearn.metrics import roc_auc_score

g_adhere_list = []
p_adhere_list = []

# Collect adherence values
for predicted, ground_truth in zip(predicted_metrics_list, ground_truth_list):
    g_adhere_list.append(ground_truth["ground_truth"]["Adherence"])
    p_adhere_list.append(predicted["predicted_metrics"]["Adherence"])

# Handle boolean values directly
mapping = {True: 1, False: 0}
y_true_numeric_alt = [mapping[val] for val in g_adhere_list]
y_pred_numeric_alt = [mapping[val] for val in p_adhere_list]

# Compute AUC-ROC
auc_roc = roc_auc_score(y_true_numeric_alt, y_pred_numeric_alt)
print("AUC-ROC:", auc_roc)

AUC-ROC: 0.75


# **Adherence Score**

In [ ]:
# Adherence score is nothing but the cosine similarity between the context
# passed vs the response generated

def adherence_score(context, response):
  context_embedding = embedder.encode(question)
  response_embedding = embedder.encode(answer)
  similarity = np.dot(context_embedding, response_embedding) /
        (np.linalg.norm(context_embedding) * np.linalg.norm(response_embedding))
  return similarity



In [ ]:
# Print Results
print("Ground Truth Values (JSON):")
print(json.dumps(ground_truth, indent=4))
print("\nPredicted Metrics:")
print(json.dumps(predicted_metrics, indent=4))
print("\nEvaluation Metrics (RMSE and AUC-ROC):")
print(json.dumps(evaluation_metrics, indent=4))

In [ ]:
# newly added on 1st Feb (not yet tested) to be used in summarization
print("Adherence Score:", adherence_score(context, answer))

# **RAG System Evaluation on RGB Dataset**

In [ ]:
# Clone the repository
!git clone https://github.com/chen700564/RGB.git
%cd RGB

# Additional LLM libraries (if required)
!pip install transformers sentence-transformers


Cloning into 'RGB'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 50 (delta 22), reused 34 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 8.16 MiB | 9.00 MiB/s, done.
Resolving deltas: 100% (22/22), done.
/content/RGB


In [ ]:
! pip install langchain
! pip install groq
! pip install -q langchain langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 775.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 9.6 MB/s eta 0:00:00


In [ ]:
import json
import langchain
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

chat = ChatGroq(temperature=0.3, groq_api_key="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", model_name="llama3-8b-8192")

prompt=ChatPromptTemplate.from_template(
      """
      Please provide a response to the query below, strictly adhering to the
      information presented in the following documents.
      Do not generate any text beyond what is explicitly stated in the documents.

      Context: {context}

      Question: {query}

      Answer:
      """
    )
chain = prompt | chat

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


# **RAG prompt to LLM as given in the research paper 239_01431** #

In [ ]:
system_prompt = """
  You are an accurate and reliable AI assistant that can
answer questions with the help of external documents.
Please note that external documents may contain noisy
or factually incorrect information. If the information in
the document contains the correct answer, you will give
an accurate answer. If the information in the document
does not contain the answer, you will generate ’I can not
answer the question because of the insufficient
information in documents.‘ If there are inconsistencies
with the facts in some of the documents, please generate
the response 'There are factual errors in the provided
documents.' and provide the correct answer.
"""
user_prompt = """
Document:\n{context} \n\nQuestion:\n{query}
"""



In [ ]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_groq import ChatGroq

system_message = SystemMessagePromptTemplate.from_template(system_prompt)
user_message = HumanMessagePromptTemplate.from_template(user_prompt)

chat_prompt = ChatPromptTemplate.from_messages([system_message, user_message])

In [ ]:
from google.colab import userdata
api_key = userdata.get('fist_key')
#api_key = "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"
api_key

'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'

In [ ]:
chat = ChatGroq(temperature=0.3, groq_api_key=api_key, model_name="llama3-8b-8192")
chain = chat_prompt | chat

In [ ]:

# Noise Robustness Evaluation

input_file = "/content/RGB/data/en_refine.json"
output_file = "/content/RGB/data/en_refine_fixed.json"

with open(input_file, "r") as f:
    lines = f.readlines()

# Wrap each line in an array
json_objects = [line.strip() for line in lines if line.strip()]
wrapped_json = f"[{','.join(json_objects)}]"

# Write to a new file
with open(output_file, "w") as f:
    f.write(wrapped_json)

# Load the fixed JSON file
with open(output_file, "r") as f:
    data = json.load(f)

for query in data[:5]:
    context = ""
    noisy_query = query["query"]
    expected_answer = query["answer"]

    pos_docs = " ".join(query["positive"]) if isinstance(query["positive"], list) else query["positive"]
    neg_docs = " ".join(query["negative"]) if isinstance(query["negative"], list) else query["negative"]

    pos = sent_tokenize(pos_docs)
    neg = sent_tokenize(neg_docs)

    context = "\n".join(pos[:3])

    context = context.join(neg[:2])

    print("context>>>>>",context)
    groq_response_noise = chain.invoke({"context": context, "query": query})

    print(f"Query: {noisy_query}\nResponse: {groq_response_noise.content}\nExpected: {expected_answer}\n")


context>>>>> Feb 10, 2022                                 Runtime:                                   2h 0m                                Carole King                                                                                          Self                                                             James Taylor                                                                                          Self                                                             Danny Kortchmar                                                                                          Self                                                             Russ Kunkel                                                                                          Self                                                             Lee Sklar                                                                                          Self                                                             Frank Marshall               

APIStatusError: Error code: 413 - {'error': {'message': 'Request too large for model `llama3-8b-8192` in organization `org_01jehtjqc2f3ft3rhykwt26n21` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Requested 6025, please reduce your message size and try again. Visit https://console.groq.com/docs/rate-limits for more information.', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}

In [ ]:
# Negative Rejection Evaluation

for query in data[5:10]:
    context = ""
    irrelevant_query = query["query"]
    expected_answer = query["answer"]

    neg_docs = "\n".join(query["negative"]) if isinstance(query["negative"], list) else query["negative"]

    neg = sent_tokenize(neg_docs)

    context = "\n".join(pos[:5])

    print("context>>>>>",context)
    groq_response_negetive = chain.invoke({"context": context, "query": irrelevant_query})

    print(f"Query: {irrelevant_query}\nResponse: {groq_response_negetive.content}\nExpected: {expected_answer}\n")


context>>>>> However, the concert tour took place in honor of the 40th anniversary.
The two might have aged since they first performed together but neither Carole King nor James Taylor have lost a beat in all these years!The concert film includes the following songs:(You Make Me Feel Like) A Natural WomanSomething in the Way She MovesSo Far AwayCarolina in My MindCountry RoadSmackwater JackWhere You Lead (lyrics changed up as the city they’re playing in replaces New York)Your Smiling FaceBeautifulShower The PeopleWay Over YonderSweet Baby James (this kicks off the second half of the film)Up on the RoofIt’s Too LateFire and RainI Feel the Earth MoveYou’ve Got a FriendHow Sweet It Is (To Be Loved by You)You Can Close Your EyesMexico (end credits)DIRECTOR: Frank MarshallFEATURING: Carole King, James Taylor, Danny Kortchmar, Peter Asher, Russ Kunkel, Leland SklarADDITIONAL MUSICIANS: Andrea Zonn, Arnold McCuller, Kate Markowitz, Robbie KondorCarole King & James Taylor: Just Call Out My Nam

In [ ]:
# Information Integration Evaluation
from sentence_transformers import SentenceTransformer, util

input_file = "/content/RGB/data/en_int.json"
output_file = "/content/RGB/data/en_int_fixed.json"

with open(input_file, "r") as f:
    lines = f.readlines()

# Wrap each line in an array
json_objects = [line.strip() for line in lines if line.strip()]
wrapped_json = f"[{','.join(json_objects)}]"

# Write to a new file
with open(output_file, "w") as f:
    f.write(wrapped_json)

# Load the fixed JSON file
with open(output_file, "r") as f:
    data = json.load(f)

retriever = SentenceTransformer("all-MiniLM-L6-v2")

for query in data[:5]:
    integrated_query = query["query"]
    expected_answer = query["answer"]

    pos_docs = [doc for doc_list in query["positive"] for doc in doc_list]

    # The positive docs already have relevant data why sort again and pick docs?
    # docs_embeddings = retriever.encode(pos_docs, convert_to_tensor=True)
    # query_embedding = retriever.encode(integrated_query, convert_to_tensor=True)
    # scores = util.pytorch_cos_sim(query_embedding, docs_embeddings)
    # scores = scores.flatten()
    # print(f">>>>>> Scores: {scores}")
    # top_docs = [pos_docs[i] for i in scores.argsort(descending=True)]

    context = "\n".join(pos_docs)
    # response = model(context, max_length=50)[0]["generated_text"]
    print("context>>>>>",context)
    groq_response_integrated = chain.invoke({"context": context, "query": integrated_query})

    print(f"Query: {integrated_query}\nResponse: {groq_response_integrated.content}\nExpected: {expected_answer}\n")


In [ ]:
# Counter Factual Robusiness evaluation

input_file = "/content/RGB/data/en_fact.json"
output_file = "/content/RGB/data/en_fact_fixed.json"

with open(input_file, "r") as f:
    lines = f.readlines()

# Wrap each line in an array
json_objects = [line.strip() for line in lines if line.strip()]
wrapped_json = f"[{','.join(json_objects)}]"

# Write to a new file
with open(output_file, "w") as f:
    f.write(wrapped_json)

# Load the fixed JSON file
with open(output_file, "r") as f:
    data = json.load(f)


for query in data[:5]:  # Limiting to first 5 queries for testing
    counterfactual_query = query["query"]
    expected_behavior = query["answer"]
    fake_answer = query["fakeanswer"]
    context = query["positive_wrong"]

    response = chain.invoke({"context": context, "query": counterfactual_query})
    #response = model(counterfactual_query, max_length=50)[0]["generated_text"]
    print(f"Query: {counterfactual_query}\nResponse: {response}\nExpected: {expected_behavior}\nFake Answer: {fake_answer}\n")


Query: Super Bowl 2021 location
Response: content='Based on the provided document, the answer to the question is:\n\nRaymond James Stadium in Glendale, Arizona.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 275, 'total_tokens': 299, 'completion_time': 0.02, 'prompt_time': 0.046140257, 'queue_time': 0.019813660000000004, 'total_time': 0.066140257}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_a97cfe35ae', 'finish_reason': 'stop', 'logprobs': None} id='run-af72c747-53dc-4093-8be6-dcdb6f4822eb-0' usage_metadata={'input_tokens': 275, 'output_tokens': 24, 'total_tokens': 299}
Expected: Tampa, Florida
Fake Answer: Glendale, Arizona

Query: Which country won the most medals at the 2018 Winter Olympics?
Response: content='According to the provided document, the United States won the most medals at the 2018 Winter Olympics, with a total of 39 medals.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 30